In [29]:
import pandas as pd
import numpy as np

Service-req files:

In [244]:
assets_df = pd.read_csv("./fetched_data/v_assets.csv")

requests_df = pd.read_csv("./fetched_data/v_requests.csv")

req_with_activities = pd.read_csv("./fetched_data/v_requests_with_activities.csv")

In [120]:
assets_df.columns

Index(['Customer', 'Country', 'Location Alt Id', 'Location Id',
       'Position Alt Id', 'positionId', 'Asset Alt Id', 'assetId',
       'Asset Description', 'positionName', 'manufacturer', 'model',
       'serialNumber'],
      dtype='object')

In [121]:
requests_df.columns

Index(['requestId', 'requestAlternateId', 'workType', 'requestDescription',
       'requestCreatedDate', 'requestTargetCompletionDate',
       'requestCompletionDate', 'serviceClassificationId',
       'serviceClassificationAlternateId', 'serviceClassificationPath',
       'locationId', 'locationAlternateId', 'locationPath', 'assetId',
       'assetAlternateId', 'assetDescription', 'completionNotes', 'customer',
       'country', 'isSelfAssign', 'priorityCode'],
      dtype='object')

In [122]:
req_with_activities.columns

Index(['requestId', 'requestAlternateId', 'workType', 'requestDescription',
       'requestCreatedDate', 'requestTargetCompletionDate',
       'requestCompletionDate', 'activityId', 'activityAlternateId',
       'activityDescription', 'activityStartDate', 'activityCompletionDate',
       'completionNotes', 'providertype', 'customer', 'country'],
      dtype='object')

In [60]:
req_with_activities = req_with_activities[['requestAlternateId','providertype','activityAlternateId','activityDescription']]

requests_df = requests_df[['requestId','assetAlternateId','customer', 'country','locationAlternateId', 'locationPath',
            'isSelfAssign','requestCreatedDate','requestDescription',
            'requestAlternateId', 'completionNotes', 'requestTargetCompletionDate',
            'serviceClassificationAlternateId', 'serviceClassificationPath', 'workType','requestCompletionDate','priorityCode']] #createdMonth, createdYear, sla, isCompleted

assets_df = assets_df[['Asset Description', 'Asset Alt Id', 'manufacturer',
           'model', 'serialNumber']]#is_HVAC, asset_type, suggested_asset

In [61]:
combined_df = requests_df.merge(assets_df, left_on =['assetAlternateId'], right_on = ['Asset Alt Id'], how='left')

combined_df = combined_df.merge(req_with_activities, on=['requestAlternateId'], how='left')

combined_df.columns#.to_csv(self.target_file_path, index=False)

Index(['requestId', 'assetAlternateId', 'customer', 'country',
       'locationAlternateId', 'locationPath', 'isSelfAssign',
       'requestCreatedDate', 'requestDescription', 'requestAlternateId',
       'completionNotes', 'requestTargetCompletionDate',
       'serviceClassificationAlternateId', 'serviceClassificationPath',
       'workType', 'requestCompletionDate', 'priorityCode',
       'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'providertype', 'activityAlternateId',
       'activityDescription'],
      dtype='object')

In [62]:
combined_df[['requestCreatedDate', 'requestTargetCompletionDate', 'requestCompletionDate']]

,requestCreatedDate,requestTargetCompletionDate,requestCompletionDate
0,2024-08-25 10:22:53.000,2024-09-03 23:00:00.000,2024-08-28 12:00:00.000
1,2024-08-25 10:22:53.000,2024-09-03 23:00:00.000,2024-08-28 12:00:00.000
2,2024-08-25 10:22:53.000,2024-09-03 23:00:00.000,2024-08-28 12:00:00.000
3,2024-08-25 10:22:53.000,2024-09-03 23:00:00.000,2024-08-28 12:00:00.000
4,2024-08-25 10:22:53.000,2024-09-03 23:00:00.000,2024-08-28 12:00:00.000
...,...,...,...
1131948,2024-05-21 20:43:15.000,2025-01-02 01:00:00.000,2025-04-15 19:11:19.769
1131949,2024-05-21 20:43:15.000,2025-01-02 01:00:00.000,2025-04-15 19:11:19.769
1131950,2024-05-21 20:43:15.000,2025-01-02 01:00:00.000,2025-04-15 19:11:19.769
1131951,2024-05-21 20:43:15.000,2025-01-02 01:00:00.000,2025-04-15 19:11:19.769


In [63]:
def to_local_datetime(date_col):
    
    if date_col is None:
        return None
    
    if date_col.isna().all():
        return date_col
    
    dt_series = pd.to_datetime(date_col, format='mixed')
    formatted = dt_series.dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
    
    return formatted.str.replace(' ', 'T')


def process_service_requests(df_service_request):
    """Efficiently process service request DataFrame"""
    
    
    date_cols = ['requestCreatedDate', 'requestTargetCompletionDate', 'requestCompletionDate']
    
    for col in date_cols:
        if col in df_service_request.columns:
            df_service_request[col] = to_local_datetime(df_service_request[col])
    
    
    df_service_request['createdYear'] = pd.to_datetime(df_service_request['requestCreatedDate']).dt.year
    df_service_request['createdMonth'] = pd.to_datetime(df_service_request['requestCreatedDate']).dt.month
    
    
    df_service_request['isCompleted'] = df_service_request['requestCompletionDate'].notna()
    
    
    conditions = [
        df_service_request['requestCompletionDate'].isna(),
        df_service_request['requestTargetCompletionDate'].isna(),
        df_service_request['requestCompletionDate'] <= df_service_request['requestTargetCompletionDate'],
        df_service_request['requestCompletionDate'] > df_service_request['requestTargetCompletionDate']
    ]
    
    choices = ['Open', 'Open', 'Met', 'Miss']
    
    df_service_request['sla'] = np.select(conditions, choices, default='Unknown')
    
    return df_service_request

# Usage
_combined_df = process_service_requests(combined_df)

In [64]:
is_hvac_df = pd.read_csv('./data/hvac_assets/IFM_Assets_RuleBasedEngineResults(IFM_Assets_RuleBasedEngineResul).csv')
is_hvac_df.drop(columns=['Asset Description'], inplace = True)
_combined_df_with_hvac = _combined_df.merge(is_hvac_df, on='Asset Alt Id', how='left') #[['assetId', 'confidence_score_rb','rule_based_asset_name']]
_combined_df_with_hvac['is_HVAC'] = _combined_df_with_hvac['Asset Alt Id'].notna().astype('boolean')
_combined_df_with_hvac.loc[_combined_df_with_hvac['is_HVAC'] == True, 'asset_type'] = 'HVAC'


In [66]:
# _combined_df_with_hvac[_combined_df_with_hvac['Asset Description'].notna()][['Asset Description']]

,Asset Description
10,Emergency Generator - STEWART - 12V-GD-350(TXR...
12,Computer Room Air Conditioning(CRAC-12)
56,PDL-External Lighting System|Système d’éclaira...
71,EMERGENCY GENERATOR CATERPILLAR(AH-HOS-GDS-GE...
72,AHU-Blower Coil Unit - Lennox - CB19/CBH19 Ser...
...,...
1131898,RFS-Ice Machine crescent w bin - Hoshizaki - K...
1131900,"Pump, Hydronic(HWP-DR-1)"
1131901,RFS-Ice Machine/Water Dispenser - Hoshizaki - ...
1131903,fire blanket-17


In [67]:
suggested_asset_df = pd.read_csv('./data/asset_suggest_data/asset_suggest_model.csv')
suggested_asset_df.rename(columns={'asset_id': 'suggested_asset'}, inplace=True)
suggested_asset_df

,id,request_id,suggested_asset,api_call_times,created_date,modified_date
0,ae0b52a6-7287-425a-a770-a7861d710fca,fc588142-140c-4e5f-9a09-7ef7b112c7c8,bd1a2f90-3a10-490b-ba9b-7259121c5dea,1,2025-08-29 09:09:08.766,2025-08-29 09:09:39.030
1,e810aaf1-4c6b-4d83-888f-f5ea8aceed7b,60d68b7e-eb81-4e15-a255-cace4582d730,c54a1010-0305-471e-a75b-9bf668f75425,1,2025-08-29 13:23:24.674,2025-08-29 13:23:46.526
2,75d4873f-7a9d-4dba-ab4f-157ed8c668b5,d81150f7-e70d-4711-bd7e-eab64b704dc4,bf5c7716-bf40-45f6-bb65-980299271128,1,2025-08-28 19:23:18.662,2025-08-28 19:24:06.442
3,af4d7f47-ebd6-479b-af6f-a7a327811cb5,07ae9d69-d004-41c3-8ef3-759dffe3dee4,866e4739-b724-4b8e-beca-f03616f63701,1,2025-08-29 13:22:56.371,2025-08-29 13:23:08.434
4,2bdf4d37-b26e-4942-92ee-f765b61c6940,b5ff3287-0500-4b89-b93f-dc2f1e306807,3e851373-ce76-4661-8248-3105fda97e0b,1,2025-08-28 23:36:05.992,2025-08-28 23:36:26.567
...,...,...,...,...,...,...
132,00d83627-c1b4-405e-833a-a98a0dcf0d0c,d5fef22e-77dc-4526-bd64-3cb709e72a19,61e62975-de8a-41d6-979c-a0f946084553,2,2025-10-09 15:19:41.627,2025-10-09 15:21:07.168
133,14a79994-32f2-4153-85a7-d06781225ebc,9b832db8-ac4d-47be-9b54-a4906def4bc2,b9128a89-e00f-430e-b2cc-4ec8f83938c6,1,2025-10-09 00:11:39.588,2025-10-09 00:11:46.181
134,853946b2-6a4a-46a7-9fd2-5500e796b5de,516f802a-85f8-48f2-a382-6b846c13a37a,7ce6e492-567d-454b-bf22-2767cfd7bb59,8,2025-10-09 14:44:07.192,2025-10-09 14:45:27.976
135,9314c609-682a-48ea-8ad0-d3ce52c459ca,ab11aede-b344-49f1-a710-73736d34b229,3fb9cf99-30e3-4d7c-9a78-af6cafc4597e,5,2025-10-09 18:19:54.824,2025-10-09 18:21:09.379


In [69]:

_combined_df_with_hvac_and_assets = _combined_df_with_hvac.merge(suggested_asset_df, left_on='requestId', right_on='request_id', how='left')

In [71]:
_combined_df_with_hvac_and_assets.columns

Index(['requestId', 'assetAlternateId', 'customer', 'country',
       'locationAlternateId', 'locationPath', 'isSelfAssign',
       'requestCreatedDate', 'requestDescription', 'requestAlternateId',
       'completionNotes', 'requestTargetCompletionDate',
       'serviceClassificationAlternateId', 'serviceClassificationPath',
       'workType', 'requestCompletionDate', 'priorityCode',
       'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'providertype', 'activityAlternateId',
       'activityDescription', 'createdYear', 'createdMonth', 'isCompleted',
       'sla', 'Customer', 'Country', 'Location Alt Id', 'Location Id',
       'Position Alt Id', 'Position Id', 'Asset Id', 'Position Name',
       'Manufacturer', 'Model', 'Serial Number', 'rule_based_asset_name',
       'confidence_score_rb', 'explanation_rb', 'is_HVAC', 'asset_type', 'id',
       'request_id', 'suggested_asset', 'api_call_times', 'created_date',
       'modified_date'],
      dtype='

In [76]:
# activity data with vendor info:
vendor_data = pd.read_csv('./data/asset_vendor/request_act_vendor.csv')
vendor_data = vendor_data[['requestAlternateId','vendorName', 'vendorAddress1', 'vendorCity',
       'vendorRegion', 'vendorCountry', 'vendorPostalCode']]
_combined_df_with_hvac_and_assets_and_vendor_info = _combined_df_with_hvac_and_assets.merge(vendor_data, on='requestAlternateId', how='left') 
_combined_df_with_hvac_and_assets_and_vendor_info.columns

Index(['requestId', 'assetAlternateId', 'customer', 'country',
       'locationAlternateId', 'locationPath', 'isSelfAssign',
       'requestCreatedDate', 'requestDescription', 'requestAlternateId',
       'completionNotes', 'requestTargetCompletionDate',
       'serviceClassificationAlternateId', 'serviceClassificationPath',
       'workType', 'requestCompletionDate', 'priorityCode',
       'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'providertype', 'activityAlternateId',
       'activityDescription', 'createdYear', 'createdMonth', 'isCompleted',
       'sla', 'Customer', 'Country', 'Location Alt Id', 'Location Id',
       'Position Alt Id', 'Position Id', 'Asset Id', 'Position Name',
       'Manufacturer', 'Model', 'Serial Number', 'rule_based_asset_name',
       'confidence_score_rb', 'explanation_rb', 'is_HVAC', 'asset_type', 'id',
       'request_id', 'suggested_asset', 'api_call_times', 'created_date',
       'modified_date', 'vendorName', 

In [85]:
_combined_df_with_hvac_and_assets_and_vendor_info

,requestId,assetAlternateId,customer,country,locationAlternateId,locationPath,isSelfAssign,requestCreatedDate,requestDescription,requestAlternateId,...,suggested_asset,api_call_times,created_date,modified_date,vendorName,vendorAddress1,vendorCity,vendorRegion,vendorCountry,vendorPostalCode
0,00002621-237b-42b7-a127-02c628ec5eae,NaN,DIRECTV,US,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ...",False,2024-08-25T10:22:53.000,S45965 - Rounds:>walking rounds no log entry,R102377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00002621-237b-42b7-a127-02c628ec5eae,NaN,DIRECTV,US,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ...",False,2024-08-25T10:22:53.000,S45965 - Rounds:>walking rounds no log entry,R102377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00002621-237b-42b7-a127-02c628ec5eae,NaN,DIRECTV,US,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ...",False,2024-08-25T10:22:53.000,S45965 - Rounds:>walking rounds no log entry,R102377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00002621-237b-42b7-a127-02c628ec5eae,NaN,DIRECTV,US,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ...",False,2024-08-25T10:22:53.000,S45965 - Rounds:>walking rounds no log entry,R102377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00002621-237b-42b7-a127-02c628ec5eae,NaN,DIRECTV,US,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ...",False,2024-08-25T10:22:53.000,S45965 - Rounds:>walking rounds no log entry,R102377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132208,fffff4a3-34e0-46a1-8b72-bde8f03972e1,NaN,Southern California Edison,US,LUS297692,"US, CA, Rosemead / General Off - Bldg 3 / 2nd ...",False,2024-05-21T20:43:15.000,C291002 - General Building Maintenance:> Mark ...,R101001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1132209,fffff4a3-34e0-46a1-8b72-bde8f03972e1,NaN,Southern California Edison,US,LUS297692,"US, CA, Rosemead / General Off - Bldg 3 / 2nd ...",False,2024-05-21T20:43:15.000,C291002 - General Building Maintenance:> Mark ...,R101001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1132210,fffff4a3-34e0-46a1-8b72-bde8f03972e1,NaN,Southern California Edison,US,LUS297692,"US, CA, Rosemead / General Off - Bldg 3 / 2nd ...",False,2024-05-21T20:43:15.000,C291002 - General Building Maintenance:> Mark ...,R101001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1132211,fffff4a3-34e0-46a1-8b72-bde8f03972e1,NaN,Southern California Edison,US,LUS297692,"US, CA, Rosemead / General Off - Bldg 3 / 2nd ...",False,2024-05-21T20:43:15.000,C291002 - General Building Maintenance:> Mark ...,R101001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
_combined_df_with_hvac_and_assets_and_vendor_info[_combined_df_with_hvac_and_assets_and_vendor_info['Location Alt Id'].notna()][['customer','Customer','Location Alt Id']]

,customer,Customer,Location Alt Id
83,Christus Health,Christus Health,LUS233225
90,Christus Health,Christus Health,LUS212034
108,Home Depot Corp,Home Depot Corp,LUS182123
134,Christus Health,Christus Health,LUS227259
138,Alaska Airlines,Alaska Airlines,LUS200048
...,...,...,...
1132062,DIRECTV,DIRECTV,LUS351961
1132078,State Farm,State Farm,LUS253870
1132098,Home Depot Corp,Home Depot Corp,LUS182434
1132135,Christus Health,Christus Health,LUS216636


______________

In [ ]:
# activity_df = _combined_df_with_hvac_and_assets_and_vendor_info[['activityDescription', 'activityAlternateId', 'providertype']]
# asset_df = _combined_df_with_hvac_and_assets_and_vendor_info[['Asset Description', 'assetAlternateId', 'manufacturer', 'model', 'serialNumber',
#             'is_HVAC', 'asset_type', 'suggested_asset', 'vendorName', 'vendorAddress1', 'vendorCity',
#             'vendorRegion', 'vendorCountry', 'vendorPostalCode']]
# country_df = _combined_df_with_hvac_and_assets_and_vendor_info[['country']]
# customer_df = _combined_df_with_hvac_and_assets_and_vendor_info[['customer']]
# location_df = _combined_df_with_hvac_and_assets_and_vendor_info[['locationAlternateId', 'locationPath']]
# service_req_df = _combined_df_with_hvac_and_assets_and_vendor_info[['createdYear', 'createdMonth', 'isSelfAssign', 'priorityCode', 
#                   'requestCreatedDate', 'requestDescription', 'requestAlternateId', 'completionNotes', 
#                   'requestTargetCompletionDate', 'serviceClassificationAlternateId', 'serviceClassificationPath',  
#                   'sla', 'requestCompletionDate', 'isCompleted', 'workType']]


In [256]:


requests_df = pd.read_csv("./fetched_data/v_requests.csv")

req_with_activities = pd.read_csv("./fetched_data/v_requests_with_activities.csv")

In [ ]:
# Activity:

In [126]:
activity_df = req_with_activities[req_with_activities['activityAlternateId'].notna()][['providertype','activityAlternateId','activityDescription']]


In [213]:
activity_df.drop_duplicates(inplace=True)

In [233]:
activity_df

,providerType,activityId,activityDescription
1,NaN,AC441457,General Mechanical | General - WRMC-EMERGENCY ...
2,TECHNICIAN,AC515192,Plumbing | General - Room N320 toilet lead is ...
3,TECHNICIAN,AC381918,"HVAC | General - THD0006 | D3032.01.136, D3032..."
6,TECHNICIAN,AC206261,General Building | Building Services - running...
7,TECHNICIAN,AC517131,Electrical | General - FR-OCF-Electric Deliver...
...,...,...,...
468822,DISPATCHSITE,AC329668,HVAC | General - ADT7821601 | Condenser PUY-A2...
468823,DISPATCHSITE,AC329658,HVAC | General - ADT7821601 | Condenser PUY-A2...
468824,DISPATCHSITE,AC329662,HVAC | General - ADT7821601 | Condenser PUY-A2...
468825,TECHNICIAN,AC154528,Cleaning Services | General - Clean warehouse ...


In [232]:
activity_df.rename(columns={'activityAlternateId': 'activityId', 'providertype':'providerType'}, inplace=True)

In [ ]:
# Asset:

In [257]:
requests_df = pd.read_csv("./fetched_data/v_requests.csv")
requests_df.columns

Index(['requestId', 'requestAlternateId', 'workType', 'requestDescription',
       'requestCreatedDate', 'requestTargetCompletionDate',
       'requestCompletionDate', 'serviceClassificationId',
       'serviceClassificationAlternateId', 'serviceClassificationPath',
       'locationId', 'locationAlternateId', 'locationPath', 'assetId',
       'assetAlternateId', 'assetDescription', 'completionNotes', 'customer',
       'country', 'isSelfAssign', 'priorityCode'],
      dtype='object')

In [258]:
requests_subset = requests_df[['requestId','assetAlternateId','requestAlternateId']]
requests_subset = requests_subset[requests_subset.assetAlternateId.notna()]

In [259]:
assets_df = pd.read_csv("./fetched_data/v_assets.csv")

In [260]:
assets_df.columns

Index(['Customer', 'Country', 'Location Alt Id', 'Location Id',
       'Position Alt Id', 'positionId', 'Asset Alt Id', 'assetId',
       'Asset Description', 'positionName', 'manufacturer', 'model',
       'serialNumber'],
      dtype='object')

In [261]:
v_assets = assets_df[['assetId','Asset Alt Id', 'Asset Description', 'manufacturer', 'model', 'serialNumber']]
v_assets = v_assets.merge(requests_subset, left_on = 'Asset Alt Id', right_on = 'assetAlternateId', how= 'left')
v_assets

,assetId,Asset Alt Id,Asset Description,manufacturer,model,serialNumber,requestId,assetAlternateId,requestAlternateId
0,bf420915-50ca-4999-9388-0e46b94f0194,AUS208832,Teller stools(Teller Stools),No Manufacturer Data,No Model Data,UNK82505,NaN,NaN,NaN
1,b8566f9d-0407-4542-8bcb-68815b89be7e,AUS168004,Quarterly HVAC Equipment Inspection(Quarterly ...,No Manufacturer Data,No Model Data,UNK21677,NaN,NaN,NaN
2,1e8c3724-864e-434d-8867-1dc843fa1898,AUS424994,Ford Pickup(Ford Truck),Ford,Pickup504197,F1,NaN,NaN,NaN
3,7a644146-e695-43fb-9e0c-edda9df5ef85,AUS208522,Condenser Unit(CU-02),TRANE,TWA090D30RAA,142527CMYA,NaN,NaN,NaN
4,032407c1-638b-4247-b16f-0589ec3ab23c,AUS211332,Irrigation System - Unknown # of Zones/Not Ass...,No Manufacturer Data,No Model Data,UNK211284,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
641842,57bc150c-1872-4912-89ff-37d7ae6a5b55,AFR113472,TR5-Heat pump|Pompe à chaleur RDC bas PAC-042,UNKNOWN,Unknown,Unknown 425,NaN,NaN,NaN
641843,221867a3-3612-4918-81cd-86ca1e2f355c,AUS218488,Exterior Doors - Other(Exterior Doors - Other),NaN,NaN,UNK49963,NaN,NaN,NaN
641844,f6daa43b-8e94-4380-89b8-4e71ef805c18,AUS306621,Fire Extinguisher - Not Specified - 5lb Class ...,Other,FireExt5lbABC,WK-650490,NaN,NaN,NaN
641845,9e4542b7-09da-4549-927d-72ca7ff3252c,AFR112004,FRSC-FEX-Extincteur n°41 - Poudre 2Kg - BA VEH...,DESAUTEL,PP2P,GYORA,NaN,NaN,NaN


In [262]:
is_hvac_df = pd.read_csv('./data/hvac_assets/IFM_Assets_RuleBasedEngineResults(IFM_Assets_RuleBasedEngineResul).csv')


In [263]:
is_hvac_df['is_HVAC'] = True
is_hvac_df.drop(columns=['Asset Description'], inplace = True)
v_assets_with_hvac = v_assets.merge(is_hvac_df, on='Asset Alt Id', how='left') #[['assetId', 'confidence_score_rb','rule_based_asset_name']]
# v_assets_with_hvac['is_HVAC'] = v_assets_with_hvac['Asset Alt Id'].notna().astype('boolean')
v_assets_with_hvac.loc[v_assets_with_hvac['is_HVAC'] == True, 'asset_type'] = 'HVAC'


In [270]:
v_assets_with_hvac.columns

Index(['assetId', 'Asset Alt Id', 'Asset Description', 'manufacturer', 'model',
       'serialNumber', 'requestId', 'assetAlternateId', 'requestAlternateId',
       'Customer', 'Country', 'Location Alt Id', 'Location Id',
       'Position Alt Id', 'Position Id', 'Asset Id', 'Position Name',
       'Manufacturer', 'Model', 'Serial Number', 'rule_based_asset_name',
       'confidence_score_rb', 'explanation_rb', 'is_HVAC', 'asset_type'],
      dtype='object')

In [271]:
final_assets_df = v_assets_with_hvac[['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'is_HVAC', 'asset_type', 'requestId','assetAlternateId','requestAlternateId']] #, 'suggested_asset'


In [272]:
final_assets_df['is_HVAC'] = final_assets_df['is_HVAC'].fillna(False)

/var/folders/r2/5_8vrrd90lsdfnwbdxcdvqxw0000gp/T/ipykernel_49403/3099090697.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_assets_df['is_HVAC'] = final_assets_df['is_HVAC'].fillna(False)
/var/folders/r2/5_8vrrd90lsdfnwbdxcdvqxw0000gp/T/ipykernel_49403/3099090697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_assets_df['is_HVAC'] = final_assets_df['is_HVAC'].fillna(False)


In [273]:
suggested_asset_df = pd.read_csv('./data/asset_suggest_data/asset_suggest_model.csv')
suggested_asset_df.rename(columns={'asset_id': 'suggested_asset'}, inplace=True)
suggested_asset_df_subset = suggested_asset_df[['request_id', 'suggested_asset']]
suggested_asset_df_subset

,request_id,suggested_asset
0,fc588142-140c-4e5f-9a09-7ef7b112c7c8,bd1a2f90-3a10-490b-ba9b-7259121c5dea
1,60d68b7e-eb81-4e15-a255-cace4582d730,c54a1010-0305-471e-a75b-9bf668f75425
2,d81150f7-e70d-4711-bd7e-eab64b704dc4,bf5c7716-bf40-45f6-bb65-980299271128
3,07ae9d69-d004-41c3-8ef3-759dffe3dee4,866e4739-b724-4b8e-beca-f03616f63701
4,b5ff3287-0500-4b89-b93f-dc2f1e306807,3e851373-ce76-4661-8248-3105fda97e0b
...,...,...
132,d5fef22e-77dc-4526-bd64-3cb709e72a19,61e62975-de8a-41d6-979c-a0f946084553
133,9b832db8-ac4d-47be-9b54-a4906def4bc2,b9128a89-e00f-430e-b2cc-4ec8f83938c6
134,516f802a-85f8-48f2-a382-6b846c13a37a,7ce6e492-567d-454b-bf22-2767cfd7bb59
135,ab11aede-b344-49f1-a710-73736d34b229,3fb9cf99-30e3-4d7c-9a78-af6cafc4597e


In [274]:
final_assets_df.columns

Index(['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'is_HVAC', 'asset_type', 'requestId',
       'assetAlternateId', 'requestAlternateId'],
      dtype='object')

In [275]:
final_assets_df = final_assets_df.merge(suggested_asset_df_subset, left_on = 'requestId', right_on = 'request_id', how = 'left')


In [276]:
final_assets_df.columns

Index(['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'is_HVAC', 'asset_type', 'requestId',
       'assetAlternateId', 'requestAlternateId', 'request_id',
       'suggested_asset'],
      dtype='object')

In [277]:
final_assets_df = final_assets_df[['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'is_HVAC', 'asset_type', 'suggested_asset','requestAlternateId']]

In [279]:
vendor_data.columns

Index(['requestAlternateId', 'requestDescription', 'activityAlternateId',
       'activityDescription', 'vendorName', 'vendorAddress1', 'vendorCity',
       'vendorRegion', 'vendorCountry', 'vendorPostalCode'],
      dtype='object')

In [ ]:
vendor_data = pd.read_csv('./data/asset_vendor/request_act_vendor.csv')
vendor_data = vendor_data[['requestAlternateId','vendorName', 'vendorAddress1', 'vendorCity',
                           'vendorRegion', 'vendorCountry', 'vendorPostalCode']]

In [286]:
assets_with_vendors = final_assets_df.merge(vendor_data, on = 'requestAlternateId', how = 'left')

In [289]:
assets_with_vendors.columns

Index(['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'is_HVAC', 'asset_type', 'suggested_asset',
       'requestAlternateId', 'vendorName', 'vendorAddress1', 'vendorCity',
       'vendorRegion', 'vendorCountry', 'vendorPostalCode'],
      dtype='object')

In [290]:
assets_with_vendors[['Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
       'serialNumber', 'is_HVAC', 'asset_type', 'suggested_asset','vendorName', 'vendorAddress1', 'vendorCity',
       'vendorRegion', 'vendorCountry', 'vendorPostalCode']]

,Asset Description,Asset Alt Id,manufacturer,model,serialNumber,is_HVAC,asset_type,suggested_asset,vendorName,vendorAddress1,vendorCity,vendorRegion,vendorCountry,vendorPostalCode
0,Teller stools(Teller Stools),AUS208832,No Manufacturer Data,No Model Data,UNK82505,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Quarterly HVAC Equipment Inspection(Quarterly ...,AUS168004,No Manufacturer Data,No Model Data,UNK21677,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ford Pickup(Ford Truck),AUS424994,Ford,Pickup504197,F1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Condenser Unit(CU-02),AUS208522,TRANE,TWA090D30RAA,142527CMYA,True,HVAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Irrigation System - Unknown # of Zones/Not Ass...,AUS211332,No Manufacturer Data,No Model Data,UNK211284,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641849,TR5-Heat pump|Pompe à chaleur RDC bas PAC-042,AFR113472,UNKNOWN,Unknown,Unknown 425,True,HVAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641850,Exterior Doors - Other(Exterior Doors - Other),AUS218488,NaN,NaN,UNK49963,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641851,Fire Extinguisher - Not Specified - 5lb Class ...,AUS306621,Other,FireExt5lbABC,WK-650490,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641852,FRSC-FEX-Extincteur n°41 - Poudre 2Kg - BA VEH...,AFR112004,DESAUTEL,PP2P,GYORA,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 'assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
#                                            'serialNumber', 'is_HVAC', 'asset_type', 'suggested_asset'
final_assets_df.drop(columns=['assetId'], inplace = True)
final_assets_df.rename(columns={'Asset Alt Id': 'assetId', 'Asset Description':'assetDescription'}, inplace=True)

In [ ]:
final_assets_df

,assetDescription,assetId,manufacturer,model,serialNumber,is_HVAC,asset_type,suggested_asset
0,Teller stools(Teller Stools),AUS208832,No Manufacturer Data,No Model Data,UNK82505,False,NaN,NaN
1,Quarterly HVAC Equipment Inspection(Quarterly ...,AUS168004,No Manufacturer Data,No Model Data,UNK21677,False,NaN,NaN
2,Ford Pickup(Ford Truck),AUS424994,Ford,Pickup504197,F1,False,NaN,NaN
3,Condenser Unit(CU-02),AUS208522,TRANE,TWA090D30RAA,142527CMYA,True,HVAC,NaN
4,Irrigation System - Unknown # of Zones/Not Ass...,AUS211332,No Manufacturer Data,No Model Data,UNK211284,False,NaN,NaN
...,...,...,...,...,...,...,...,...
641490,TR5-Heat pump|Pompe à chaleur RDC bas PAC-042,AFR113472,UNKNOWN,Unknown,Unknown 425,True,HVAC,NaN
641491,Exterior Doors - Other(Exterior Doors - Other),AUS218488,NaN,NaN,UNK49963,False,NaN,NaN
641492,Fire Extinguisher - Not Specified - 5lb Class ...,AUS306621,Other,FireExt5lbABC,WK-650490,False,NaN,NaN
641493,FRSC-FEX-Extincteur n°41 - Poudre 2Kg - BA VEH...,AFR112004,DESAUTEL,PP2P,GYORA,False,NaN,NaN


In [ ]:
requests_subset = self.df_request[['requestId','assetAlternateId', 'requestAlternateId']]
requests_subset = requests_subset[requests_subset.assetAlternateId.notna()]

v_assets = self.df_assets[['assetId','Asset Alt Id', 'Asset Description', 'manufacturer', 'model', 'serialNumber']]
v_assets = v_assets.merge(requests_subset, left_on = 'Asset Alt Id', right_on = 'assetAlternateId', how= 'left')

is_hvac_df = self.is_hvac_df.copy()
is_hvac_df['is_HVAC'] = True
is_hvac_df.drop(columns=['Asset Description'], inplace = True)
v_assets_with_hvac = v_assets.merge(is_hvac_df, on='Asset Alt Id', how='left')
v_assets_with_hvac.loc[v_assets_with_hvac['is_HVAC'] == True, 'asset_type'] = 'HVAC'

final_assets_df = v_assets_with_hvac[['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
                                        'serialNumber', 'is_HVAC', 'asset_type', 'requestId','assetAlternateId']]
final_assets_df.loc[:, 'is_HVAC'] = final_assets_df['is_HVAC'].fillna(False)

suggested_asset_df = self.suggested_asset_df.copy()
suggested_asset_df.rename(columns={'asset_id': 'suggested_asset'}, inplace=True)
suggested_asset_df_subset = suggested_asset_df[['request_id', 'suggested_asset']]

final_assets_df = final_assets_df.merge(suggested_asset_df_subset, left_on = 'requestId', right_on = 'request_id', how = 'left')
assets_df = final_assets_df[['assetId', 'Asset Description', 'Asset Alt Id', 'manufacturer', 'model',
                                    'serialNumber', 'is_HVAC', 'asset_type', 'suggested_asset']]

In [228]:
# LOCATED_AT = 
requests_df[['assetAlternateId','locationAlternateId']].drop_duplicates().dropna()

,assetAlternateId,locationAlternateId
1,AUS396115,LUS409154
3,AUS277136,LUS182335
7,AFR114227,LFR109610
9,AUS289127,LUS225461
10,AUS326295,LUS235133
...,...,...
454098,AGB154668,LGB110361
454123,AGB170735,LGB114963
454142,AGB160516,LGB112472
454159,AUS351724,LUS349545


In [219]:
requests_df[['country']].drop_duplicates()

,country
0,US
7,FR
12,GB
19,IT
23,CH
29,MX
63,IN
66,HK
78,PK
105,VN


In [101]:
country_df.drop_duplicates()

,country
0,US
56,FR
74,GB
87,IT
91,CH
100,MX
207,IN
221,HK
240,PK
291,VN


In [220]:
requests_df[['customer']].drop_duplicates()

,customer
0,DIRECTV
1,Christus Health
3,Home Depot Corp
4,S&P Global
7,Sidel
8,Southern California Edison
12,NEC
14,State Farm
16,MUFG
17,Clorox


In [102]:
customer_df.drop_duplicates()

,customer
0,DIRECTV
10,Christus Health
12,Home Depot Corp
13,S&P Global
56,Sidel
57,Southern California Edison
74,NEC
82,State Farm
84,MUFG
85,Clorox


In [222]:
requests_df[['locationAlternateId', 'locationPath']].drop_duplicates()

,locationAlternateId,locationPath
0,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ..."
1,LUS409154,"US,TX,Texarkana / ALT-T CHRISTUS Pine Street -..."
2,LUS226145,"US,TX,Texarkana / ALT-T St. Michael Acute Hosp..."
3,LUS182335,"US, GA, Atlanta / Store Support Ctr - Bldg D /..."
4,LUS364066,"US, NY, New York / 55 Water Street / Floor 40 ..."
...,...,...
454149,LUS375530,"US, SC, Myrtle Beach / 3694 Palmeto Point Blv_..."
454156,LUS232880,"US,TX,Tyler / TMF-T Louis & Peaches Owen Heart..."
454158,LUS264832,"US, CA, Gorman / Bailey Sub / 1st Floor / Buil..."
454184,LUS373084,"US, TX, Irving / 8880 Esters Blv_TX-0047 / Flo..."


In [104]:
location_df.drop_duplicates()

,locationAlternateId,locationPath
0,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ..."
10,LUS409154,"US,TX,Texarkana / ALT-T CHRISTUS Pine Street -..."
11,LUS226145,"US,TX,Texarkana / ALT-T St. Michael Acute Hosp..."
12,LUS182335,"US, GA, Atlanta / Store Support Ctr - Bldg D /..."
13,LUS364066,"US, NY, New York / 55 Water Street / Floor 40 ..."
...,...,...
1131896,LUS375530,"US, SC, Myrtle Beach / 3694 Palmeto Point Blv_..."
1131921,LUS232880,"US,TX,Tyler / TMF-T Louis & Peaches Owen Heart..."
1131923,LUS264832,"US, CA, Gorman / Bailey Sub / 1st Floor / Buil..."
1131989,LUS373084,"US, TX, Irving / 8880 Esters Blv_TX-0047 / Flo..."


In [236]:
# locationAlternateId,locationPath locationPath
location_df.rename(columns={'locationAlternateId': 'locationId'}, inplace=True)

/var/folders/r2/5_8vrrd90lsdfnwbdxcdvqxw0000gp/T/ipykernel_49403/1751191557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_df.rename(columns={'locationAlternateId': 'locationId'}, inplace=True)


In [237]:
location_df

,locationId,locationPath
0,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ..."
1,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ..."
2,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ..."
3,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ..."
4,LUS351642,"US, CO, CASTLE ROCK / 5454 GARTON RD - CRBC03 ..."
...,...,...
1132208,LUS297692,"US, CA, Rosemead / General Off - Bldg 3 / 2nd ..."
1132209,LUS297692,"US, CA, Rosemead / General Off - Bldg 3 / 2nd ..."
1132210,LUS297692,"US, CA, Rosemead / General Off - Bldg 3 / 2nd ..."
1132211,LUS297692,"US, CA, Rosemead / General Off - Bldg 3 / 2nd ..."


In [210]:
# service_req_df.drop_duplicates()
temp_ser_req = requests_df[['isSelfAssign', 'priorityCode', 
                  'requestCreatedDate', 'requestDescription', 'requestAlternateId', 'completionNotes', 
                  'requestTargetCompletionDate', 'serviceClassificationAlternateId', 'serviceClassificationPath',  
                  'requestCompletionDate', 'workType']]
# 'createdYear', 'createdMonth', 'sla', 'isCompleted'

In [211]:
def to_local_datetime(date_col):
    
    if date_col is None:
        return None
    
    if date_col.isna().all():
        return date_col
    
    dt_series = pd.to_datetime(date_col, format='mixed')
    formatted = dt_series.dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
    
    return formatted.str.replace(' ', 'T')


def process_service_requests(df_service_request):
    """Efficiently process service request DataFrame"""
    
    
    date_cols = ['requestCreatedDate', 'requestTargetCompletionDate', 'requestCompletionDate']
    
    for col in date_cols:
        if col in df_service_request.columns:
            df_service_request[col] = to_local_datetime(df_service_request[col])
    
    
    df_service_request['createdYear'] = pd.to_datetime(df_service_request['requestCreatedDate']).dt.year
    df_service_request['createdMonth'] = pd.to_datetime(df_service_request['requestCreatedDate']).dt.month
    
    
    df_service_request['isCompleted'] = df_service_request['requestCompletionDate'].notna()
    
    
    conditions = [
        df_service_request['requestCompletionDate'].isna(),
        df_service_request['requestTargetCompletionDate'].isna(),
        df_service_request['requestCompletionDate'] <= df_service_request['requestTargetCompletionDate'],
        df_service_request['requestCompletionDate'] > df_service_request['requestTargetCompletionDate']
    ]
    
    choices = ['Open', 'Open', 'Met', 'Miss']
    
    df_service_request['sla'] = np.select(conditions, choices, default='Unknown')
    
    return df_service_request

# Usage
service_req_df = process_service_requests(temp_ser_req)

/var/folders/r2/5_8vrrd90lsdfnwbdxcdvqxw0000gp/T/ipykernel_49403/293563969.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_service_request[col] = to_local_datetime(df_service_request[col])
/var/folders/r2/5_8vrrd90lsdfnwbdxcdvqxw0000gp/T/ipykernel_49403/293563969.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_service_request[col] = to_local_datetime(df_service_request[col])
/var/folders/r2/5_8vrrd90lsdfnwbdxcdvqxw0000gp/T/ipykernel_49403/293563969.py:23: SettingWithCopyWarning: 
A value is 

In [238]:
service_req_df

,isSelfAssign,priorityCode,requestCreatedDate,requestDescription,requestAlternateId,completionNotes,requestTargetCompletionDate,serviceClassificationAlternateId,serviceClassificationPath,requestCompletionDate,workType,createdYear,createdMonth,isCompleted,sla
0,False,NaN,2024-08-25T10:22:53.000,S45965 - Rounds:>walking rounds no log entry,R102377,NaN,2024-09-03T23:00:00.000,SCDTV100575,General Facilities Services | General Facility...,2024-08-28T12:00:00.000,NaN,2024,8,True,Met
1,False,APM,2025-08-08T16:29:46.241,WRMC-EMERGENCY GENERATOR (DIESEL CHECK) (WEEK...,RCHR206889,NaN,2025-07-29T04:59:00.000,SCCHR100813,Preventive Maintenance | General PM,2025-08-11T14:09:40.070,PM,2025,8,True,Miss
2,False,P7,2025-09-18T15:10:41.085,"Room N320 toilet lead is broken, patient in th...",RCHR233869,NaN,2025-10-09T15:09:33.000,SCCHR100097,Plumbing | Fixtures | Toilet (Patient Room/Pub...,NaN,Reactive,2025,9,False,Open
3,False,P10,2025-08-01T01:44:31.058,"THD0006 | D3032.01.136, D3032.01.1412 | 3425, ...",RTHD110282,NaN,2025-09-01T03:59:00.000,SCTHD100526,Preventive Maintenance | Preventive Maintenanc...,NaN,PM,2025,8,False,Open
4,False,NaN,2024-09-19T18:14:07.000,C1048688 - General Facility Service Requests:>...,R100018,NaN,2024-09-26T19:14:07.000,SCSPG100926,General Facility Services | General Facility S...,2024-09-25T23:28:04.211,NaN,2024,9,True,Met
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454267,False,P7,2025-02-10T11:33:54.447,Clean warehouse floor,RTHD105185,Cleaned warehouse,2025-03-12T11:33:53.000,SCTHD100170,Janitorial | Cleaning | Floor Clean General,2025-02-10T13:37:47.683,Corrective,2025,2,True,Met
454268,False,P14,2025-03-12T03:41:18.630,วันที่ 12 Mar ช่วงบ่าย / ส่งเอกสารมอบอำนาจ ที่...,R110704,NaN,2025-03-19T03:41:17.000,SCSYN100137,Office Services | Mail Room | Deliver/Pick-up ...,2025-03-13T02:54:21.273,Reactive,2025,3,True,Met
454269,False,P4,2025-09-04T12:28:13.910,Light out on the third floor at the top of the...,RABV107470,NaN,2025-09-29T19:27:12.000,SCABV100239,Maintenance & Repair | Electrical | Electrical...,NaN,Proactive,2025,9,False,Open
454270,False,P5,2025-03-21T15:03:56.926,Hall 7 Public Toilets - Replace 8.5w lamp abov...,R115699,NaN,2025-04-20T15:03:00.000,SCNEC100290,Maintenance & Repair | Electrical | Electrical...,2025-03-21T15:30:23.659,Proactive,2025,3,True,Met


In [239]:
service_req_df.rename(columns={'requestAlternateId': 'requestId', 'serviceClassificationAlternateId': 'serviceClassificationId'}, inplace=True)

/var/folders/r2/5_8vrrd90lsdfnwbdxcdvqxw0000gp/T/ipykernel_49403/3299668959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  service_req_df.rename(columns={'requestAlternateId': 'requestId', 'serviceClassificationAlternateId': 'serviceClassificationId'}, inplace=True)


In [ ]:
['isSelfAssign', 'priorityCode', 'requestCreatedDate',
       'requestDescription', 'requestId', 'completionNotes',
       'requestTargetCompletionDate', 'serviceClassificationId',
       'serviceClassificationPath', 'requestCompletionDate', 'workType',
       'createdYear', 'createdMonth', 'isCompleted', 'sla']

Index(['isSelfAssign', 'priorityCode', 'requestCreatedDate',
       'requestDescription', 'requestId', 'completionNotes',
       'requestTargetCompletionDate', 'serviceClassificationId',
       'serviceClassificationPath', 'requestCompletionDate', 'workType',
       'createdYear', 'createdMonth', 'isCompleted', 'sla'],
      dtype='object')

-----------------------

In [ ]:
def data_processor(self, df_path):

      df = pd.read_csv(df_path).iloc[10:11, :]
      #print(df)
      try:
         df_service_request = pd.DataFrame(df[['requestId','requestDescription','completionNotes','isSelfAssign','priorityCode',
                                               'workType','customer','country','locationId','serviceClassificationId',
                                               'requestCreatedDate','requestTargetCompletionDate','requestCompletionDate']])
      except:
         return("Interupted!! Service Request Features not in the input data!!!")

      # # Date formatting:
      
      # df_service_request.isSelfAssign = int(df_service_request.isSelfAssign)
      df_service_request.requestCreatedDate = df_service_request.requestCreatedDate.str.replace(" ", "T")
      df_service_request.requestTargetCompletionDate = df_service_request.requestTargetCompletionDate.str.replace(" ", "T")
      df_service_request.requestCompletionDate = df_service_request.requestCompletionDate.str.replace(" ", "T")
      
      df_service_request.loc[df_service_request.requestCreatedDate.notnull(),'createdYear'] = pd.to_datetime(df_service_request.requestCreatedDate).dt.year
      df_service_request.loc[df_service_request.requestCreatedDate.notnull(),'createdMonth'] = pd.to_datetime(df_service_request.requestCreatedDate).dt.month
      df_service_request.loc[df_service_request.requestCompletionDate.notnull(),'isCompleted'] = True

      df_service_request.loc[df_service_request.requestCompletionDate.isnull(),'sla'] = "Open"
      df_service_request.loc[df_service_request.requestTargetCompletionDate.isnull(),'sla'] = "Open" #guard
      df_service_request.loc[df_service_request.requestCompletionDate <= df_service_request.requestTargetCompletionDate,'sla'] = "Met"
      df_service_request.loc[df_service_request.requestCompletionDate > df_service_request.requestTargetCompletionDate,'sla'] = "Miss"

      return df_service_request

In [ ]:

def to_local_datetime(date_str):
    
    if date_str is None:
        return None
    s = str(date_str).strip()
    if not s:
        return None

    formatted = pd.to_datetime(s, format='mixed').strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

    return formatted.replace(' ', 'T')

In [ ]:
# pd.to_datetime(df['requesttargetcompletiondate']).dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
# pd.to_datetime(df['requesttargetcompletiondate']).dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
pd.to_datetime(df['requesttargetcompletiondate'],format='mixed').dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3]
# df['requesttargetcompletiondate'] = 

In [ ]:
customer_df = total_data[['customer']]
customer_df.to_csv('./total_data/customers.csv', index=False)

country_df = total_data[['country']]
country_df.to_csv('./total_data/countries.csv', index=False)

service_class_df = total_data[['serviceClassificationId', 'serviceClassificationAlternateId', 'serviceClassification']]
service_class_df.to_csv('./total_data/service_classes.csv', index=False)

activity_df = total_data[['requestId', 'requestAlternateId', 'activityId', 'activityAlternateId', 'providertype']]
activity_df.to_csv('./total_data/activities.csv', index=False)

# # asset_category_df = total_data[['requestId','assetId', 'confidence', 'is_HVAC', 'asset_type', 'suggested_asset']]
# # asset_category_df.to_csv('./total_data/asset_categories.csv', index=False)

worktype_df = total_data[['worktype']]
worktype_df.to_csv('./total_data/worktypes.csv', index=False)

location_df = total_data[['locationId', 'locationAlternateId', 'locationPath']]
location_df.to_csv('./total_data/locations.csv', index=False)

service_request_df = total_data[['requestId', 'requestAlternateId', 'requestDescription', 'completionNotes', 'requestcreateddate', 'requesttargetcompletiondate', 
                                 'requestcompletiondate', 'priorityCode', 'worktype', 'customer', 'country', 'locationId', 'locationAlternateId',
                                 'serviceClassificationId', 'serviceClassificationAlternateId', 'isSelfAssign', 'sla', 'createdYear', 'createdMonth', 'isCompleted']]
service_request_df.to_csv('./total_data/service_requests_1.csv', index=False)


asset_df = total_data[['assetId', 'assetAlternateId', 'assetDescription', 'country', 'customer', 'locationId', 'locationAlternateId',
                      'manufacturer', 'positionName', 'positionId', 'Position Alt Id', 'model', 'serialNumber']] 
asset_df.to_csv('./total_data/all_assets.csv', index=False)

asset_assigned_to_sr = total_data[['requestId', 'requestAlternateId', 'assetId', 'assetAlternateId']]
asset_assigned_to_sr.to_csv('./total_data/sr_assetsassigned.csv', index=False)



In [ ]:
# activity data with vendor info:

activity_df = pd.read_csv('./total_data/activities.csv')
vendor_data = pd.read_csv('./vendor_data_source/request_act_vendor.csv')


In [ ]:
activities_data_with_vendor_info = activity_df.merge(vendor_data, on='requestAlternateId', how='left') 
activities_data_with_vendor_info.to_csv('./total_data/activities.csv', index=False)

In [ ]:
is_hvac_df = pd.read_csv('./hvac_feature_source/IFM_Assets_RuleBasedEngineResults(IFM_Assets_RuleBasedEngineResul).csv')

In [ ]:
total_df_with_hvac = total_data.merge(is_hvac_df, left_on='assetAlternateId', right_on='Asset Alt Id', how='left') #[['assetId', 'confidence_score_rb','rule_based_asset_name']]

In [ ]:
total_df_with_hvac['is_HVAC'] = total_df_with_hvac['Asset Alt Id'].notna().astype('boolean')


In [ ]:
# total_df_with_hvac['asset_type'] = 
total_df_with_hvac.loc[total_df_with_hvac['is_HVAC'] == True, 'asset_type'] = 'HVAC'

In [ ]:
suggested_asset_df = pd.read_csv('./hvac_feature_source/asset_suggest_model.csv')
suggested_asset_df.rename(columns={'asset_id': 'suggested_asset'}, inplace=True)

In [ ]:
total_df_with_hvac_and_assets = total_df_with_hvac.merge(suggested_asset_df, left_on='requestId', right_on='request_id', how='left')

In [ ]:
asset_categories = total_df_with_hvac_and_assets[['requestId', 'requestAlternateId', 'assetId', 'assetAlternateId','suggested_asset', 'is_HVAC','asset_type']]#.rename(columns={'assetAlternateId': 'assetId'})
asset_categories.to_csv('./total_data/asset_categories.csv', index=False)

In [ ]:
priority_df = pd.read_csv('./priority_code_source/priority_codes 7.csv')
priority_df
priority_df.to_csv('./total_data/priority_codes.csv', index=False)